# Multi-parametric global optimization

This tutorial shows how to find global minimum of multivariable function calculated by Modelica model. Cells are evaluated by pressing `Shift-Enter`.

## 1. PyFmi library

Import pyfmi library:

In [ ]:
from pyfmi import load_fmu

if error happens, you need to install pyfmi, e.g. `conda install -c conda-forge pyfmi.`

## 2. Simulate FMU in Python

The objective function to be minimized is saved in file CostFunction.mo in the same directory as this notebook. The goal is to find values for the three parameters x, y, and z that minimize the value of the variable v (which is constant over time). The generated CostFunction.fmu file is stored in the same directory. You can create your own Modelica model with another objective function. Menu item File -> FMI -> Export FMU opens the dialog to generated fmu in OpenModelica OMEdit.

Now load the FMU:

In [ ]:
model = load_fmu('CostFunction.fmu')

Create default simulation options, we will set it further

In [ ]:
opts = model.simulate_options()

Define the function that runs the model with different parameter values:

In [ ]:
def simulateWithParam(x,y,z,sopts=opts):
    model.reset()
    model.instantiate()
    model.set('x',x)
    model.set('y',y)
    model.set('z',z)
    res = model.simulate(final_time=1, options = sopts)
    return res

Check the simulation with given parameter values

In [ ]:
r1 = simulateWithParam(0,0,0)

Display the time array from results:

In [ ]:
r1['time']

Define the function to plot results:

In [ ]:
def plot(x,y,z=None,labelx=None,labely=None,labelz=None):
    %matplotlib inline
    import matplotlib.pyplot as plt
    fig = plt.figure()
    plt.plot(x,y,'r',label=labelx)
    if z is not None:
        plt.plot(x,z,'b')
    plt.show()


Make sure the "v" is constant:

In [ ]:
plot(r1['time'],r1['v'])

Display the "v" array:

In [ ]:
r1['v']

## 3. Define the objective
Define the objective function in the format required by the optimization algorithm:

In [ ]:
def objFun(p):
    r = simulateWithParam(p[0],p[1],p[2],sopts=opts)
    return r['v'][-1]
    

Test the objective function:

In [ ]:
objFun([0.0,0.0,0.0])

## 4. The optimization

Import the scipi.optimize module:

In [ ]:
import scipy.optimize as optim


Run the optimization basinhopping algorithm with initial parameter values x=0, y=0, z=0 and maximally 100 iterations. It take several minutes to finish:

In [ ]:
ret = optim.basinhopping(objFun, [0,0,0], niter = 100)

Print the full optimizatin output class:

In [ ]:
ret

Print the parameter and bojective values:

In [ ]:
print("x = ", ret.x[0], "\ny = ", ret.x[1], "\nz = ", ret.x[2], "\nobjective = ", ret.fun)

## 5. Assignment:
 - extend the objective function so that there is one extra parameter that is also involved in the optimization
 - use a different algorithm (e.g. differential evolution)for the optimization from the *Global optimization* list at [docs.scipy.org/doc/scipy/reference/optimize.html](docs.scipy.org/doc/scipy/reference/optimize.html)

In [ ]:
ret2 = optim.differential_evolution(objFun, [[-10,10],[-10,10],[-10,10]])